In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# ============================================================
# Step 1: Image Standardization Function
# ============================================================
def standardize_image(img, target_size=(224, 224)):
    """Resize, reorder color channels, and normalize image."""
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, target_size)
    img_normalized = img_resized / 255.0
    img_standardized = (img_normalized - 0.5) / 0.5
    return img_standardized.astype(np.float32)


# ============================================================
# Step 2: Load & Standardize Dataset
# ============================================================
def load_and_standardize_dataset(base_dir, classes, target_size=(224, 224)):
    """
    Loads all images from the dataset, applies standardization,
    and returns NumPy arrays (X, y).
    """
    X, y = [], []
    for idx, label in enumerate(classes):
        folder = os.path.join(base_dir, label)
        if not os.path.exists(folder):
            print(f"⚠️ Folder not found: {folder}")
            continue

        print(f"\n📂 Processing class: {label}")
        for file in tqdm(os.listdir(folder)):
            img_path = os.path.join(folder, file)
            try:
                img = cv2.imread(img_path)
                if img is None:
                    continue
                processed = standardize_image(img, target_size)
                X.append(processed)
                y.append(idx)
            except Exception as e:
                print(f"❌ Error with {img_path}: {e}")

    X = np.array(X)
    y = np.array(y)
    print(f"\n✅ All classes processed successfully!")
    print(f"Total images: {len(X)}")
    return X, y


# ============================================================
# Step 3: Run on DermalScan Dataset
# ============================================================
data_dir = '/content/gdrive/MyDrive/DERMALSCAN/DATASET'
classes = ['wrinkles', 'dark spots', 'puffy eyes', 'clear skin']

X, y = load_and_standardize_dataset(data_dir, classes)

print(f"\n✅ Dataset Ready for Model Training")
print(f"Images Shape: {X.shape}")
print(f"Labels Shape: {y.shape}")
print(f"Pixel Range: {X.min()} to {X.max()}")



📂 Processing class: wrinkles


100%|██████████| 300/300 [00:02<00:00, 118.60it/s]



📂 Processing class: dark spots


100%|██████████| 301/301 [00:06<00:00, 46.92it/s]



📂 Processing class: puffy eyes


100%|██████████| 300/300 [00:06<00:00, 44.33it/s]



📂 Processing class: clear skin


100%|██████████| 300/300 [00:07<00:00, 42.64it/s]



✅ All classes processed successfully!
Total images: 1201

✅ Dataset Ready for Model Training
Images Shape: (1201, 224, 224, 3)
Labels Shape: (1201,)
Pixel Range: -1.0 to 1.0


In [5]:
np.save('/content/gdrive/MyDrive/DERMALSCAN/X_standardized.npy', X)
np.save('/content/gdrive/MyDrive/DERMALSCAN/y_labels.npy', y)
print("💾 Saved standardized dataset successfully!")


💾 Saved standardized dataset successfully!


In [6]:
import cv2
import numpy as np
from tqdm import tqdm
import os

# ============================================================
# Step 2.1: DullRazor Algorithm — Hair Removal
# ============================================================
def remove_hair_dullrazor(img):
    """
    Removes thin dark hair strands using morphological operations and inpainting.
    1. Convert to grayscale.
    2. Apply black-hat morphological operation to detect hair.
    3. Create a binary mask of hair regions.
    4. Use inpainting to fill hair pixels with nearby skin texture.
    """
    gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)

    # Detect hair with morphological black-hat operation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)

    # Threshold to create a hair mask
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)

    # Inpaint hair regions with neighboring pixels
    inpainted = cv2.inpaint((img * 255).astype(np.uint8), mask, 1, cv2.INPAINT_TELEA)
    return inpainted.astype(np.float32) / 255.0


# ============================================================
# Step 2.2: Noise Reduction — Median & Gaussian Filtering
# ============================================================
def remove_noise(img):
    """
    Reduces small noise and artifacts from the image.
    Uses median filter (for salt-pepper noise) and
    Gaussian blur (for smooth surface appearance).
    """
    median_filtered = cv2.medianBlur((img * 255).astype(np.uint8), 3)
    gaussian_filtered = cv2.GaussianBlur(median_filtered, (3, 3), 0)
    return gaussian_filtered.astype(np.float32) / 255.0


# ============================================================
# Step 2.3: Combined Preprocessing Function
# ============================================================
def preprocess_hair_and_noise(img):
    """
    Combined pipeline:
    1. Hair removal (DullRazor)
    2. Noise reduction
    """
    no_hair = remove_hair_dullrazor(img)
    cleaned = remove_noise(no_hair)
    return cleaned


# ============================================================
# Step 2.4: Apply to Entire Dataset
# ============================================================
def apply_hair_noise_removal(X):
    """
    Applies hair and noise removal to all standardized images.
    """
    processed_images = []
    print("\n🧴 Applying Hair & Noise Removal to Dataset...")
    for i in tqdm(range(len(X))):
        processed = preprocess_hair_and_noise(X[i])
        processed_images.append(processed)
    return np.array(processed_images, dtype=np.float32)


# ============================================================
# Example Usage (After Step 1)
# ============================================================
# Assuming you already have X, y loaded from Step 1
# X, y = load_and_standardize_dataset(data_dir, classes)

X_clean = apply_hair_noise_removal(X)
print("✅ Hair & Noise Removal Completed!")
print("Processed dataset shape:", X_clean.shape)



🧴 Applying Hair & Noise Removal to Dataset...


100%|██████████| 1201/1201 [00:23<00:00, 50.32it/s]


✅ Hair & Noise Removal Completed!
Processed dataset shape: (1201, 224, 224, 3)


In [7]:
np.save('/content/gdrive/MyDrive/DERMALSCAN/X_clean.npy', X_clean)
print("💾 Hair & Noise Removed Dataset Saved Successfully!")


💾 Hair & Noise Removed Dataset Saved Successfully!


In [8]:
import cv2
import numpy as np
from tqdm import tqdm

# ============================================================
# Step 3.1: Apply CLAHE to Improve Local Contrast
# ============================================================
def apply_clahe(img):
    """
    Enhances local contrast in skin areas using CLAHE.
    Works on the L-channel (lightness) in LAB color space.
    """
    lab = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Create CLAHE object (clip limit prevents over-contrast)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)

    merged = cv2.merge((l_clahe, a, b))
    enhanced = cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)
    return enhanced.astype(np.float32) / 255.0


# ============================================================
# Step 3.2: Gamma Correction — Exposure Normalization
# ============================================================
def gamma_correction(img, gamma=1.2):
    """
    Adjusts overall brightness by gamma exponent.
    gamma > 1 → brightens image, gamma < 1 → darkens.
    """
    corrected = np.power(img, 1.0 / gamma)
    return np.clip(corrected, 0, 1)


# ============================================================
# Step 3.3: White Balance Normalization
# ============================================================
def white_balance(img):
    """
    Normalizes color channels to correct color cast.
    Based on Gray-World assumption: avg(R)=avg(G)=avg(B).
    """
    result = img.copy()
    mean_r = np.mean(result[:, :, 0])
    mean_g = np.mean(result[:, :, 1])
    mean_b = np.mean(result[:, :, 2])
    mean_gray = (mean_r + mean_g + mean_b) / 3

    result[:, :, 0] = np.clip(result[:, :, 0] * (mean_gray / mean_r), 0, 1)
    result[:, :, 1] = np.clip(result[:, :, 1] * (mean_gray / mean_g), 0, 1)
    result[:, :, 2] = np.clip(result[:, :, 2] * (mean_gray / mean_b), 0, 1)

    return result


# ============================================================
# Step 3.4: Combined Correction Pipeline
# ============================================================
def color_illumination_correction(img):
    """
    Sequentially applies:
      1. CLAHE  – local contrast enhancement
      2. Gamma  – exposure adjustment
      3. White-balance normalization
    """
    clahe_img = apply_clahe(img)
    gamma_img = gamma_correction(clahe_img, gamma=1.2)
    balanced_img = white_balance(gamma_img)
    return balanced_img


# ============================================================
# Step 3.5: Apply to Entire Dataset
# ============================================================
def apply_color_correction(X):
    """
    Applies color & illumination correction to every image in dataset.
    """
    corrected_images = []
    print("\n🎨 Applying Color & Illumination Correction...")
    for i in tqdm(range(len(X))):
        corrected = color_illumination_correction(X[i])
        corrected_images.append(corrected)
    return np.array(corrected_images, dtype=np.float32)


# ============================================================
# Example Usage (after Step 2)
# ============================================================
# X_clean = apply_hair_noise_removal(X)
X_color_corrected = apply_color_correction(X_clean)
print("✅ Color & Illumination Correction Complete!")
print("Processed dataset shape:", X_color_corrected.shape)



🎨 Applying Color & Illumination Correction...


100%|██████████| 1201/1201 [00:08<00:00, 136.19it/s]


✅ Color & Illumination Correction Complete!
Processed dataset shape: (1201, 224, 224, 3)


In [9]:
np.save('/content/gdrive/MyDrive/DERMALSCAN/X_color_corrected.npy', X_color_corrected)
print("💾 Saved color-corrected dataset successfully!")


💾 Saved color-corrected dataset successfully!


In [10]:
import cv2
import numpy as np
from tqdm import tqdm

# ============================================================
# Step 4.1: Skin Mask Extraction (Using YCrCb + Otsu)
# ============================================================
def extract_skin_mask(img):
    """
    Extracts a binary mask of skin regions using color space filtering and Otsu's thresholding.
    Steps:
    1. Convert image to YCrCb color space (good for skin detection).
    2. Define skin color range.
    3. Apply mask to isolate skin tones.
    4. Apply Otsu’s thresholding for automatic segmentation.
    """
    ycrcb = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2YCrCb)
    lower = np.array([0, 133, 77], dtype=np.uint8)
    upper = np.array([255, 173, 127], dtype=np.uint8)
    skin_mask = cv2.inRange(ycrcb, lower, upper)

    # Apply Otsu’s thresholding to refine mask
    _, otsu_mask = cv2.threshold(skin_mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_mask


# ============================================================
# Step 4.2: Morphological Refinement
# ============================================================
def refine_mask(mask):
    """
    Refines binary mask using morphological operations.
    - Dilation: fills small holes in skin areas.
    - Closing: removes small black gaps inside white regions.
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    refined = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    refined = cv2.morphologyEx(refined, cv2.MORPH_DILATE, kernel, iterations=1)
    return refined


# ============================================================
# Step 4.3: Crop the Largest Connected Skin Region
# ============================================================
def crop_largest_region(img, mask):
    """
    Finds the largest contour (main skin area) and crops around it.
    """
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return img  # fallback: return original image

    # Select largest skin region
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    cropped = img[y:y+h, x:x+w]
    cropped_resized = cv2.resize(cropped, (224, 224))
    return cropped_resized


# ============================================================
# Step 4.4: Combined Skin Segmentation Pipeline
# ============================================================
def skin_segmentation_pipeline(img):
    """
    Full segmentation process:
      1. Create skin mask.
      2. Refine mask using morphology.
      3. Crop the largest skin region.
    """
    mask = extract_skin_mask(img)
    refined = refine_mask(mask)
    cropped = crop_largest_region(img, refined)
    return cropped


# ============================================================
# Step 4.5: Apply to Entire Dataset
# ============================================================
def apply_skin_segmentation(X):
    """
    Applies segmentation and cropping to every image in dataset.
    """
    segmented_images = []
    print("\n🧬 Applying Skin Segmentation & Cropping...")
    for i in tqdm(range(len(X))):
        segmented = skin_segmentation_pipeline(X[i])
        segmented_images.append(segmented)
    return np.array(segmented_images, dtype=np.float32)


# ============================================================
# Example Usage (after Step 3)
# ============================================================
# X_color_corrected = apply_color_correction(X_clean)
X_segmented = apply_skin_segmentation(X_color_corrected)

print("✅ Skin Segmentation & Cropping Complete!")
print("Processed dataset shape:", X_segmented.shape)



🧬 Applying Skin Segmentation & Cropping...


100%|██████████| 1201/1201 [00:02<00:00, 548.05it/s]


✅ Skin Segmentation & Cropping Complete!
Processed dataset shape: (1201, 224, 224, 3)


In [11]:
np.save('/content/gdrive/MyDrive/DERMALSCAN/X_segmented.npy', X_segmented)
print("💾 Saved segmented dataset successfully!")


💾 Saved segmented dataset successfully!


In [13]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# ============================================================
# Step 5.1: Validate images and remove unreadable/corrupt ones
# ============================================================
def clean_dataset(base_dir, classes):
    """
    Verifies that all images in each class folder can be loaded correctly.
    Removes any corrupt or unreadable files.
    """
    valid_images = {cls: [] for cls in classes}
    print("🧹 Cleaning and validating dataset...")

    for cls in classes:
        class_path = os.path.join(base_dir, cls)
        for img_name in tqdm(os.listdir(class_path)):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            if img is not None and img.size > 0:
                valid_images[cls].append(img_path)
            else:
                print(f"⚠️ Corrupt or invalid image removed: {img_path}")

    return valid_images


# Example Usage:
base_dir = '/content/gdrive/MyDrive/DERMALSCAN/DATASET'
classes = ['wrinkles', 'dark spots', 'puffy eyes', 'clear skin']

clean_data = clean_dataset(base_dir, classes)
print("✅ Dataset verified and cleaned!")


🧹 Cleaning and validating dataset...


100%|██████████| 300/300 [00:03<00:00, 83.12it/s]

✅ Dataset verified and cleaned!


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_images(X, y, target_per_class):
    """
    Augments minority classes to balance dataset.
    - Rotates, flips, zooms, shifts, and adjusts brightness.
    """
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )

    X_aug, y_aug = list(X), list(y)
    class_counts = {label: y.count(label) for label in set(y)}
    print("\n⚖️ Balancing dataset with augmentation...")

    for label in set(y):
        deficit = target_per_class - class_counts[label]
        if deficit > 0:
            indices = [i for i, l in enumerate(y) if l == label]
            for i in range(deficit):
                img = X[indices[i % len(indices)]]
                aug_img = next(datagen.flow(np.expand_dims(img, 0), batch_size=1))[0]
                X_aug.append(aug_img)
                y_aug.append(label)

    return np.array(X_aug), np.array(y_aug)


In [15]:
def split_dataset(X, y, test_size=0.2, val_size=0.1):
    """
    Splits dataset into training, validation, and test sets.
    """
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(test_size + val_size), stratify=y, random_state=42)
    val_relative = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=val_relative, stratify=y_temp, random_state=42)

    print("✅ Dataset Split Complete:")
    print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
    return X_train, X_val, X_test, y_train, y_val, y_test


In [16]:
# Suppose X_segmented (from Step 4) and y_labels exist
target_count = max([list(y_labels).count(lbl) for lbl in set(y_labels)])

# Balance dataset
X_balanced, y_balanced = augment_images(X_segmented, y_labels, target_per_class=target_count)

# Split for training
X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(X_balanced, y_balanced)


NameError: name 'y_labels' is not defined

In [17]:
import cv2
import os
import numpy as np
from tqdm import tqdm

# Path to your dataset directory
base_dir = '/content/gdrive/MyDrive/DERMALSCAN/DATASET'
classes = ['wrinkles', 'dark spots', 'puffy eyes', 'clear skin']

IMG_SIZE = 224  # match EfficientNet input size

X_segmented, y_labels = [], []

print("📥 Loading and labeling dataset...")

for label, cls in enumerate(classes):
    class_path = os.path.join(base_dir, cls)
    for img_name in tqdm(os.listdir(class_path), desc=f"Processing {cls}"):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0  # normalize
            X_segmented.append(img)
            y_labels.append(label)  # 0=wrinkles, 1=dark_spots, etc.

X_segmented = np.array(X_segmented, dtype='float32')
y_labels = np.array(y_labels)

print("✅ Dataset Loaded Successfully!")
print("Images Shape:", X_segmented.shape)
print("Labels Shape:", y_labels.shape)


📥 Loading and labeling dataset...


Processing clear skin: 100%|██████████| 300/300 [00:03<00:00, 77.48it/s]


✅ Dataset Loaded Successfully!
Images Shape: (1201, 224, 224, 3)
Labels Shape: (1201,)


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

def augment_images(X, y, target_per_class=300):
    """
    Balances dataset by applying data augmentation until each class
    reaches target_per_class samples.
    """
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True
    )

    X_aug, y_aug = list(X), list(y)
    y_list = y.tolist()  # ✅ convert y to list for counting
    class_counts = {label: y_list.count(label) for label in set(y_list)}

    print("\n⚖️ Balancing dataset with augmentation...")

    for label in class_counts:
        deficit = target_per_class - class_counts[label]
        if deficit > 0:
            idxs = [i for i, lbl in enumerate(y) if lbl == label]
            for i in range(deficit):
                img = X[idxs[i % len(idxs)]]
                aug_iter = datagen.flow(np.expand_dims(img, 0), batch_size=1)
                aug_img = next(aug_iter)[0]
                X_aug.append(aug_img)
                y_aug.append(label)

    return np.array(X_aug), np.array(y_aug)


In [21]:
# Convert y_labels to list for counting
y_list = y_labels.tolist()

# Target number per class (largest class count)
target_count = max([y_list.count(lbl) for lbl in set(y_list)])

# Balance dataset using augmentation
X_balanced, y_balanced = augment_images(X_segmented, y_labels, target_per_class=target_count)

# Split the dataset
X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(X_balanced, y_balanced)

print("✅ Dataset successfully balanced and split!")
print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")



⚖️ Balancing dataset with augmentation...
✅ Dataset Split Complete:
Train: 842 | Val: 241 | Test: 121
✅ Dataset successfully balanced and split!
Training set: (842, 224, 224, 3), Validation set: (241, 224, 224, 3), Test set: (121, 224, 224, 3)


In [22]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Load base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # Freeze for feature extraction phase

# Build classifier head
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
output = Dense(4, activation='softmax')(x)  # 4 classes

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 5,124 (20.02 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [24]:
model.compile(optimizer=Adam(1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical


In [26]:
# X_train, X_val, X_test = np arrays of images
# y_train, y_val, y_test = integer labels (0, 1, 2, 3)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)


In [27]:
train_datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_gen = train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
val_gen = val_datagen.flow(X_val, y_val, batch_size=32)
test_gen = test_datagen.flow(X_test, y_test, batch_size=32)


In [28]:
# Load pretrained EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base for first stage

# Add classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,214,055 (16.08 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [29]:
checkpoint = ModelCheckpoint("dermalscan_best_model.h5", monitor="val_accuracy", save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=3, min_lr=1e-5, verbose=1)


In [31]:
history_stage1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[checkpoint, early_stop, reduce_lr],
    verbose=1
)


Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2585 - loss: 1.3994
Epoch 1: val_accuracy did not improve from 0.25311
27/27 ━━━━━━━━━━━━━━━━━━━━ 96s 4s/step - accuracy: 0.2588 - loss: 1.3993 - val_accuracy: 0.2531 - val_loss: 1.3875 - learning_rate: 0.0010
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2595 - loss: 1.3940
Epoch 2: val_accuracy did not improve from 0.25311
27/27 ━━━━━━━━━━━━━━━━━━━━ 96s 4s/step - accuracy: 0.2594 - loss: 1.3940 - val_accuracy: 0.2490 - val_loss: 1.3868 - learning_rate: 0.0010
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2802 - loss: 1.3875
Epoch 3: val_accuracy did not improve from 0.25311
27/27 ━━━━━━━━━━━━━━━━━━━━ 105s 4s/step - accuracy: 0.2792 - loss: 1.3876 - val_accuracy: 0.2490 - val_loss: 1.3874 - learning_rate: 0.0010
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2297 - loss: 1.3941
Epoch 4: val_accuracy did not improve from 0.25311
27/27 ━━━━━━━━━━━━━━━━━━━━ 95s 4s/step - accur

In [32]:
# Ensure images are in correct range
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to one-hot
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=4)
y_val = to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

print("✅ Shapes:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_val:", X_val.shape, "y_val:", y_val.shape)


✅ Shapes:
X_train: (842, 224, 224, 3) y_train: (842, 4, 4)
X_val: (241, 224, 224, 3) y_val: (241, 4, 4)
